In [9]:
import os
import pandas as pd
from dataclasses import dataclass
from pathlib import Path

from src.constants import *
from src.utils.common import read_yaml, create_directories
from src import logger 
from sklearn.linear_model import ElasticNet
import joblib

In [4]:
%pwd

'c:\\Users\\amil\\OneDrive\\Documents\\Data_Science_end_to_end_project'

In [ ]:
os.chdir("../")
%pwd

In [30]:
@dataclass
class ModelTrainerConfig:
    root_dir: Path
    train_data_path: Path
    test_data_path: Path
    model_name: str
    alpha: float
    l1_ratio: float
    target_name: str

In [31]:
class ConfigurationManager:
    def __init__(self,
                config_file_path = CONFIG_FILE_PATH,
                params_file_path = PARAMS_FILE_PATH,
                schema_file_path = SCHEMA_FILE_PATH
    ):
        self.config = read_yaml(config_file_path)
        self.params = read_yaml(PARAMS_FILE_PATH)
        self.schema = read_yaml(schema_file_path)
        
        
    def get_model_trainer_config(self)-> ModelTrainerConfig:
        config = self.config.model_trainer
        params = self.params.ElasticNet
        schema = self.schema.TARGET_COLUMN
        
        create_directories([config.root_dir])
        
        model_trainer_config = ModelTrainerConfig(
            root_dir = config.root_dir,
            train_data_path = config.train_data_path,
            test_data_path = config.test_data_path,
            model_name = config.model_name,
            alpha = params.alpha,
            l1_ratio = params.l1_ratio,
            target_name = schema.name
        )

        return model_trainer_config

In [32]:
class ModelTrainer:
    def __init__(self, config = ModelTrainerConfig):
        self.config = config
        
    def train_model(self):
        train_data = pd.read_csv(self.config.train_data_path)
        test_data = pd.read_csv(self.config.test_data_path)
        
        train_X = train_data.drop([self.config.target_name],axis= 1)
        test_X = test_data.drop([self.config.target_name], axis = 1)
        
        train_Y = train_data[[self.config.target_name]]
        test_Y = test_data[[self.config.target_name]]
        
        lr = ElasticNet(alpha = self.config.alpha, l1_ratio = self.config.l1_ratio, random_state =42)
        lr.fit(train_X,train_Y)
        
        joblib.dump(lr, os.path.join(self.config.root_dir, self.config.model_name))

In [33]:
try:
    config = ConfigurationManager()
    model_trainer_config = config.get_model_trainer_config()
    model_trainer = ModelTrainer(config = model_trainer_config)
    model_trainer.train_model()
except Exception as e:
    raise e
    

[2025-11-01 13:48:37,615:, INFO:, common:, yaml file : config\config.yaml loaded successfully]
[2025-11-01 13:48:37,618:, INFO:, common:, yaml file : params.yaml loaded successfully]
[2025-11-01 13:48:37,622:, INFO:, common:, yaml file : schema.yaml loaded successfully]
[2025-11-01 13:48:37,624:, INFO:, common:, created directory for artifacts/model_trainer]
